# Исследование надежности заемщиков


**Задача проекта**: На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок.


**Описание проекта**: На основе данных кредитного отдела банка было проведено исследование влияниевлияния семейного положения и
количества детей на факт погашения кредита в срок. Была получена информация о
данных. Определены и обработаны пропуски. Заменены типы данных на соответствующие
хранящимся данным. Удалены дубликаты. Выделены леммы в значениях столбца и
категоризированны данные, а также сделаны выводы.

## Изучение общей информациюинформации о данных

In [77]:
import pandas as pd
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

1) Выводим количество пропущенных значений для каждого столбца.

In [79]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

2)  В двух столбцах есть пропущенные значения. В столбце `total_income` хранится данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце заполним медианным значением по каждому типу из столбца `income_type`.

In [80]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться аномалии, как например отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Заменим все отрицательные значения положительными.

In [86]:
data['days_employed'] = data['days_employed'].abs()

Проверим еше раз данные столбца со стежм. Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [87]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.137274
компаньон            1555.947387
пенсионер          365286.622650
предприниматель       520.848083
сотрудник            1572.328285
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно. Нам для исследования эти типы не понадобятся - оставим данные аномальные нетронутыми.

Аномальыне значения есть и в столбце `children`.

In [88]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Удалиим строки, в которых встречаются такие аномальные значения.

In [89]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

### Заполнение пропусков

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [92]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Проверим, что пропусков не осталось.

In [93]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [63]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Выведем количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [64]:
data.duplicated().sum()

54

In [94]:
data = data.drop_duplicates()

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному. 

In [95]:
data['education'] = data['education'].str.lower()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [96]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [97]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведемна экран перечень уникальных целей взятия кредита из столбца `purpose`, чтобы посмотреть, какие категории можно выделить в данном столбце.

In [98]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

На основании данных из столбца `purpose` сформируем новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [99]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [100]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Анализ данных и ответы на вопросы исследования

**Вопрос 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [101]:
pivot_children = data.pivot_table(index = ['children'], columns = 'debt', values = 'family_status_id', aggfunc='count')
pivot_children = pivot_children.reset_index()
pivot_children['ratio'] = round(pivot_children[1] / (pivot_children[0] + pivot_children[1]), 3)
display(pivot_children)
ratio = pivot_children.loc[:,'ratio']
display("{0:.2f}% - процент должников без детей".format(ratio[0]*100))
display("{0:.2f}% - процент должников с 1м ребенком".format(ratio[1]*100))
display("{0:.2f}% - процентдолжников с 2-мя детьми".format(ratio[2]*100))
display("{0:.2f}% - процентдолжников с 3-мя детьми".format(ratio[3]*100))
display("{0:.2f}% - процент должников с 4-мя детьми".format(ratio[4]*100))

debt,children,0,1,ratio
0,0,13044.0,1063.0,0.075
1,1,4365.0,444.0,0.092
2,2,1858.0,194.0,0.095
3,3,303.0,27.0,0.082
4,4,37.0,4.0,0.098
5,5,9.0,NaN,NaN


'7.50% - процент должников без детей'

'9.20% - процент должников с 1м ребенком'

'9.50% - процентдолжников с 2-мя детьми'

'8.20% - процентдолжников с 3-мя детьми'

'9.80% - процент должников с 4-мя детьми'

**Вывод:**  В среднем по выборке можно сделать вывод, что клиенты с детьми менее надежны в выплате кредитов в срок. Однако в данной категории явно выделяются семьи с 3-мя детьми - самые добросовестные заемщики среди остальных категорий клиентов с детьми. Самые благоприятные заемщики - клиенты без детей. 
    

**Вопрос 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [102]:
pivot_status = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'family_status_id', aggfunc='count')
pivot_status = pivot_status.reset_index()
pivot_status['ratio'] = round((pivot_status[1] / (pivot_status[0] + pivot_status[1])), 3)
display(pivot_status)
ratio_st = pivot_status.loc[:,'ratio']
display("{0:.2f}% - процент должников со статусом \'не женат / не замужем\'".format(ratio_st[0]*100))
display("{0:.2f}% - процент должников со статусом \'в разводе\'".format(ratio_st[1]*100))
display("{0:.2f}% - процент должников со статусом \'вдовец / вдова\'".format(ratio_st[2]*100))
display("{0:.2f}% - процент должников со статусом \'гражданский брак\'".format(ratio_st[3]*100))
display("{0:.2f}% - процент должников со статусом \'женат /замужем\'".format(ratio_st[4]*100))

debt,family_status,0,1,ratio
0,Не женат / не замужем,2523,273,0.098
1,в разводе,1105,84,0.071
2,вдовец / вдова,888,63,0.066
3,гражданский брак,3761,385,0.093
4,женат / замужем,11339,927,0.076


"9.80% - процент должников со статусом 'не женат / не замужем'"

"7.10% - процент должников со статусом 'в разводе'"

"6.60% - процент должников со статусом 'вдовец / вдова'"

"9.30% - процент должников со статусом 'гражданский брак'"

"7.60% - процент должников со статусом 'женат /замужем'"

**Вывод:** Половина выборки - заемщики, находящиеся в браке и  возвращающие кредиты. Данная категория заемщиков является и самой надежной в вопросе возврата кредита в срок, куда относятся также одинокие люди в разводе и потерявшие супруга. Самыми ненадежными в данной категории являются незамужние и клиенты, отметившие статус своих отношений как "гражданский брак". 

**Вопрос 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [103]:
pivot_income = data.pivot_table(index = ['total_income_category'], columns = 'debt', values = 'family_status_id', aggfunc='count')
pivot_income = pivot_income.reset_index()
pivot_income['ratio'] = round((pivot_income[1] / (pivot_income[0] + pivot_income[1])), 3)
display(pivot_income)
ratio_in = pivot_income.loc[:,'ratio']
display("{0:.2f}% - процент должников с доходом 1 000 001 и выше".format(ratio_in[0]*100))
display("{0:.2f}% - процент должников с доходом от 200 001 до 1 000 000".format(ratio_in[1]*100))
display("{0:.2f}% - процент должников с доходом от 50 000 до 200 000".format(ratio_in[2]*100))
display("{0:.2f}% - процент должников с доходом 30 001 до 50 000".format(ratio_in[3]*100))
display("{0:.2f}% - процент должников с доходом до 30 000".format(ratio_in[4]*100))

debt,total_income_category,0,1,ratio
0,A,23,2,0.080
1,B,4660,354,0.071
2,C,14585,1353,0.085
3,D,328,21,0.060
4,E,20,2,0.091


'8.00% - процент должников с доходом 1 000 001 и выше'

'7.10% - процент должников с доходом от 200 001 до 1 000 000'

'8.50% - процент должников с доходом от 50 000 до 200 000'

'6.00% - процент должников с доходом 30 001 до 50 000'

'9.10% - процент должников с доходом до 30 000'

**Вывод:** Случаи невозврата кредита в срок встречаются среди клиентов каждой категории по доходам, что говорит о том, что высокий доход - не гарантия возврата кредитных средств в срок. Самая малая доля невозврата кредитов в срок наблюдается среди клиетов с доходами от 50 001 до 200 000. Самые надежные заемщики - клиенты со средним уровнем дохода (со средним и, видимо, стабильным). Самые нестабильные заемщики - с доходом от 30 000 и менее. Высокая доля должников среди клиентов с доходом от 50 000 до 200 000, чт овозможно связано с нестабильными вариантами дохода. 

**Вопрос 4. Как разные цели кредита влияют на его возврат в срок?**

In [104]:
pivot_purpose = data.pivot_table(index = ['purpose_category'], columns = 'debt', values = 'family_status_id', aggfunc='count')
pivot_purpose = pivot_purpose.reset_index()
pivot_purpose['ratio'] = round((pivot_purpose[1] / (pivot_purpose[0] + pivot_purpose[1])), 3)
display(pivot_purpose)

ratio_aim = pivot_purpose.loc[:,'ratio']
display("{0:.2f}% - процент должников с кредитом на автомобиль".format(ratio_aim[0]*100))
display("{0:.2f}% - процент должников с кредитом на недвижимость".format(ratio_aim[1]*100))
display("{0:.2f}% - процент должников с кредитом на образование".format(ratio_aim[2]*100))
display("{0:.2f}% - процент должников с кредитом на свадьбу".format(ratio_aim[3]*100))

debt,purpose_category,0,1,ratio
0,операции с автомобилем,3881,400,0.093
1,операции с недвижимостью,9974,780,0.073
2,получение образования,3620,369,0.093
3,проведение свадьбы,2141,183,0.079


'9.30% - процент должников с кредитом на автомобиль'

'7.30% - процент должников с кредитом на недвижимость'

'9.30% - процент должников с кредитом на образование'

'7.90% - процент должников с кредитом на свадьбу'

**Вывод:** Самое большое число заемщиков с невозвратами кредитов (780 человек) - по категории "операции с недвижимостью".  Но в этой  категории самое больше количество клиентов, что делает долю невозврата в этой категории самой низкой. Самая высокая доля в двух категориях: каждый девятый заемщик не возвращает в срок кредиты на образование и автомобили. 

**Вопрос 5. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:* Пропуски данных наблюдались в 2-х столбцах:`total_income` (ежемесячный доход) и `days_employed` (общий трудовой стаж в днях). Информация в данных столбцах могла отсутствовать по причине незаполнения данных клиентами банка. Свой ежемесячный доход могли не указать студенты, пенсионеры, не работающие граждане, лица на иждивении, либо же заемщики не захотели раскрывать данную информацию банку. Та же категория сотрудников могла и не указать свой трудовой стаж (по неимению его). 

**Вопрос 5. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* При заполнении пропусков количественных характеристик можно использовать среднее значение, медианное или моду, чтобы оценить типичные значения выборки.  В конкретном случае среднеквадратичное значение наиболее применимо, так как среднее будет характеризоваться большим разрывом между минимумом в выборке и максимумом, а мода в свою очередь слишком "привязывает" характеристику к какой-то одной категории клиентов банка.

## Общий вывод.

Стояла задача разобраться, влияет ли семейное положение/количество детей/уровень дохода/цель кредита на факт погашения кредита в срок. При работе с данными обнаружилось, что в них были пропущенны данные о доходах клиентов и трудовом стаже (около 1% от всех данных - были заполненны медианными значением), присутствовали дубликаты (были удалены). Также при преобработке данных для удобства анализа были выделены категории по целям кредита и по доходу. 

**По итогу можно отменить, что самыми надежными заещиками являются**:
1) *незамужние клиенты без детей или многодетные с 3-мя детьми*;
2) *находящиеся в браке или разводе, либо являющиеся вдовцами*;
3) *клиенты с доходом от 30 000 до 50 0000*;
4) *клиенты, берущие кредит для "операций с недвижимостью*".

**Наибольшая доля дожников определена среди клиентов, которые:**
1) *имеют от одного ребенка и более*;
2) *являются незамужними\холостыми или живущими в "гражданском браке*";
3) *имеют доход менее 30 000*;
4) *берут кредиты на автомобили и образование*. 